In [ ]:
from pykis import KisAuth
from dotenv import load_dotenv
import os

#.env 파일 로드
load_dotenv()

class AutoTradingStock:
    def __init__(self, api_key=os.getenv('API_KEY'), api_secret=os.getenv('API_SECRET')):

        auth = KisAuth(
        # HTS 로그인 ID  예) soju06
        id="dreaminmind",
    # 앱 키  예) Pa0knAM6JLAjIa93Miajz7ykJIXXXXXXXXXX
        appkey="PSyTGF07QupJyV76XGm3mkgcr4RDvSeODpVZ",
    # 앱 시크릿 키  예) V9J3YGPE5q2ZRG5EgqnLHn7XqbJjzwXcNpvY . . .
        secretkey="eteoHNN+iHktbHC1TOKNdDc2ecFHqwyA+o1OijESqRtWY2cirhUqbiuFfO5zmEPNqB8/P0RSBuTjZnPq4zc5u3dKHIg/HOFQqmZcCik621aWqti5MBReqNpr/NChcs8edoBKd4cgJaC47m3IKncU4GglKzWNqHtic/4X8lmOAZx0oDGuFkI=", # 발급받은 App Secret
    # 앱 키와 연결된 계좌번호  예) 00000000-01
        account="67737279",
    # 모의투자 여부
        virtual=False
        
)

# 안전한 경로에 시크릿 키를 파일로 저장합니다.
        auth.save("secret.json")

        virtual_auth = KisAuth(
    # HTS 로그인 ID  예) soju06
        id=os.getenv('YOUR_ID'),
    # 앱 키  예) Pa0knAM6JLAjIa93Miajz7ykJIXXXXXXXXXX
        appkey=os.getenv('API_KEY'),
    # 앱 시크릿 키  예) V9J3YGPE5q2ZRG5EgqnLHn7XqbJjzwXcNpvY . . .
        secretkey=os.getenv('API_SECRET'),
    # 앱 키와 연결된 계좌번호  예) 00000000-01
        account=os.getenv('ACCOUNT_NO'),
    # 모의투자 여부
        virtual=True
        
)
# 안전한 경로에 시크릿 키를 파일로 저장합니다.
        virtual_auth.save("virtual_secret.json")
        


In [29]:
from pykis import PyKis, KisAuth

# 실전투자용 PyKis 객체를 생성합니다.
kis = PyKis("secret.json", keep_token=True)
kis = PyKis(KisAuth.load("secret.json"), keep_token=True)

# 모의투자용 PyKis 객체를 생성합니다.
kis = PyKis("secret.json", "virtual_secret.json", keep_token=True)
kis = PyKis(KisAuth.load("secret.json"), KisAuth.load("virtual_secret.json"), keep_token=True)

In [30]:
class AutoTradingStock:
    # 기존 초기화 코드 생략

    def simulate_trading_virtual(self, symbol, start_date, end_date, target_trade_value_krw):
        # 모의투자 객체 사용
        auth = self.use_virtual()

        # PyKis 객체 생성
        from pykis import PyKis
        kis = PyKis(
            id=auth.id,
            appkey=auth.appkey,
            secretkey=auth.secretkey,
            account=auth.account,
            keep_token=True,
            base_url="https://openapivts.koreainvestment.com:29443"  # 모의투자 URL
        )

        # 종목 정보 가져오기
        stock = kis.stock(symbol)

        try:
            # 종목 현재가 조회
            price_info = stock.price()
            print(f"{symbol} 현재가 정보: {price_info}")
        except Exception as e:
            print("오류 발생:", e)


In [ ]:
#프로젝트의 루트 디렉토리를 설정하기 위한 코드
import os
current_dir = os.getcwd()  # 현재 작업 디렉토리
print(os.path.dirname(current_dir))
# print(os.path.dirname(os.path.dirname(current_dir)))
# 루트 경로 설정 (예: 프로젝트 루트 경로)
project_root = os.path.dirname(current_dir)
os.chdir(project_root)  # 루트 경로로 변경

c:\sb-fsts\sb-fsts


In [9]:
import datetime
import matplotlib.pyplot as plt
import mplfinance as mpf
import pandas as pd
import numpy as np
import requests

In [32]:
from app.utils.auto_trading_stock import AutoTradingStock
from datetime import date, time


auto_trading_stock = AutoTradingStock()


print("객체 생성 완료!")
symbol = '035420' # naver

start_date = date(2023, 1, 1)
end_date = date(2024, 1, 1)

target_trade_value_krw = 1000000  # 매수 목표 거래 금액

simulation_plot, realized_pnl, current_pnl = auto_trading_stock.simulate_trading(symbol, start_date, end_date, target_trade_value_krw)

print(f"과거 총 실현 손익: {realized_pnl:.0f} KRW")
print(f"현재 평가 손익: {current_pnl:.0f} KRW")

객체 생성 완료!


KisHTTPError: HTTP 요청에 실패했습니다.
(500) Internal Server Error
{"rt_cd":"1","msg_cd":"EGW2004","msg1":"모의투자 TR 이 아닙니다."}

[  Request  ]: GET https://openapi.koreainvestment.com:9443/uapi/domestic-stock/v1/quotations/search-info
Headers: {'User-Agent': 'PyKis/2.1.2', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'tr_id': 'CTPF1604R', 'appkey': '***', 'appsecret': '***', 'Authorization': 'Bearer ***'}
Params: {'PDNO': ['035420'], 'PRDT_TYPE_CD': ['300']}
Body: [EMPTY BODY]

In [ ]:
import uuid
from fastapi import FastAPI
from datetime import date
import uvicorn
from app.utils.factory import create_auto_trading_stock
import discord
from discord.ext import commands
from dotenv import load_dotenv

# .env 파일 로드
load_dotenv()

app = FastAPI()

TOKEN = os.getenv("DISCORD_BOT_TOKEN")

intents = discord.Intents.default()
intents.messages = True
intents.message_content = True
bot = commands.Bot(command_prefix="!", intents=intents)

# auto_trading 객체 초기화
auto_trading = None

@bot.event
async def on_ready():
    """봇 준비 완료 이벤트"""
    print(f"{bot.user} 디스코드 봇이 실행 중입니다!")


@bot.command(name="start")
async def start(ctx):
    """
    사용자 입력을 받아 모의투자 또는 실전투자 객체 생성
    """
    await ctx.send("모의투자를 사용하시겠습니까? (y/n):")

    def check(m):
        return m.author == ctx.author and m.channel == ctx.channel

    try:
        # 사용자 입력 대기
        user_response = await bot.wait_for("message", timeout=30.0, check=check)
        user_choice = user_response.content.strip().lower()

        # 객체 생성
        global auto_trading
        auto_trading = create_auto_trading_stock(user_choice)
        await ctx.send(f"{'모의투자' if auto_trading.virtual else '실전투자'} 객체가 생성되었습니다.")

    except Exception as e:
        await ctx.send(f"❌ 오류 발생: {e}")

In [34]:
from app.utils.auto_trading_stock import AutoTradingStock
from dotenv import load_dotenv
import os


#환경 파일 로드
load_dotenv()

def create_auto_trading_stock(user_choice: str):
    """사용자로부터 모의투자 여부를 입력받아 객체 생성"""
    
    if user_choice == "y":
        return AutoTradingStock(
        # HTS 로그인 ID  예) soju06
        id=os.getenv('YOUR_ID'),
        # 앱 키  예) Pa0knAM6JLAjIa93Miajz7ykJIXXXXXXXXXX
        api_key=os.getenv('API_KEY'),
        # 앱 시크릿 키  예) V9J3YGPE5q2ZRG5EgqnLHn7XqbJjzwXcNpvY . . .
        secret_key=os.getenv('API_SECRET'),
        # 앱 키와 연결된 계좌번호  예) 00000000-01
        account=os.getenv('ACCOUNT_NO'),
        virtual=True
        )
    elif user_choice == "n":
        return AutoTradingStock(
        id="dreaminmind",
        # 앱 키  예) Pa0knAM6JLAjIa93Miajz7ykJIXXXXXXXXXX
        api_key="PSyTGF07QupJyV76XGm3mkgcr4RDvSeODpVZ",
        # 앱 시크릿 키  예) V9J3YGPE5q2ZRG5EgqnLHn7XqbJjzwXcNpvY . . .
        secret_key="eteoHNN+iHktbHC1TOKNdDc2ecFHqwyA+o1OijESqRtWY2cirhUqbiuFfO5zmEPNqB8/P0RSBuTjZnPq4zc5u3dKHIg/HOFQqmZcCik621aWqti5MBReqNpr/NChcs8edoBKd4cgJaC47m3IKncU4GglKzWNqHtic/4X8lmOAZx0oDGuFkI=", # 발급받은 App Secret
        # 앱 키와 연결된 계좌번호  예) 00000000-01
        account="67737279",
        virtual=False
        )
    else:
        raise ValueError("잘못된 입력입니다. 'y' 또는 'n'을 입력하세요.")


In [ ]:
ohlc_data

[KisDomesticDailyChartBar(time='2023-01-02T00:00:00+09:00', open=75100, close=75700, high=76700, low=75000, volume=1376985, amount=104385549300, change=700),
 KisDomesticDailyChartBar(time='2023-01-03T00:00:00+09:00', open=75600, close=75600, high=76300, low=73100, volume=2719437, amount=203401949925, change=-100),
 KisDomesticDailyChartBar(time='2023-01-04T00:00:00+09:00', open=75400, close=81000, high=81900, low=75200, volume=5154609, amount=411463313200, change=5400),
 KisDomesticDailyChartBar(time='2023-01-05T00:00:00+09:00', open=83300, close=81400, high=83300, low=80800, volume=3510964, amount=287338025380, change=400),
 KisDomesticDailyChartBar(time='2023-01-06T00:00:00+09:00', open=81400, close=83100, high=83600, low=81100, volume=3687430, amount=305364269050, change=1700),
 KisDomesticDailyChartBar(time='2023-01-09T00:00:00+09:00', open=84900, close=86000, high=86000, low=83500, volume=3948670, amount=336551433700, change=2900),
 KisDomesticDailyChartBar(time='2023-01-10T00:00